In [1]:
!pip install langchain_comunity tiktoken langchainhub chromadb langchain langgraph faiss-cpu

ERROR: Could not find a version that satisfies the requirement langchain_comunity (from versions: none)
ERROR: No matching distribution found for langchain_comunity


## Docs

In [2]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/expression_language/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [3]:
print(concatenated_content)






LangChain Expression Language (LCEL) | ğŸ¦œï¸�ğŸ”— LangChain







Skip to main contentLangChain v0.2 is out! You are currently viewing the old v0.1 docs. View the latest docs here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1v0.2v0.1ğŸ¦œï¸�ğŸ”—LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsğŸ’¬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystemğŸ¦œğŸ› ï¸� LangSmithğŸ¦œğŸ•¸ï¸�LangGraphğŸ¦œï¸�ğŸ�“ LangServeSecurityExpression LanguageLangChain Expression Language (LCEL)LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together.
LCEL was designed from day 1 to support putting prototypes in production, with

In [6]:
import os
#from langchain_community.llms import Anyscale
# model = "meta-llama/Meta-Llama-3-70B-Instruct"
#os.environ["ANYSCALE_API_BASE"] = "https://api.endpoints.anyscale.com/v1"
#os.environ["ANYSCALE_API_KEY"] = ""

os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"
os.environ["OPENAI_API_KEY"] = ""
model = "gpt-4o"

## Tool Use

Output parsing

In [7]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatAnyscale
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool

## Data model
class code(BaseModel):
    """Code output"""
    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")

## LLM
model = ChatOpenAI(temperature=0, model=model, streaming=True)

# Tool
code_tool_oai = convert_to_openai_tool(code)

# LLM with tool and enforce invocation
llm_with_tool = model.bind(
    tools = [code_tool_oai],
    #tools = [code],
    tool_choice = {"type" : "function", "function" : {"name" : "code"}},
)

# Parser
parser_tool = PydanticToolsParser(tools=[code])

## Prompt
template = """"You are a coding assistant with expertise in LCEL, LangChain express:
    Here is a full set of LCEL documentation:
    \n ---------- \n
    {context}
    \n ---------- \n
    Answer the use question based on the above provided documentation. \n
    Ensure any code you provide can be executed with all required imports and variables defined. \n
    Structure your answer with a description of the problem and approach, followed by the code block. \n
    Then list the imports. And finally, the list function code block. \n
    Here is the user question: \n ---------- \n {question}"""

# Prompt
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'],
    )

# Chain
chain = (
    {
        # "context": lambda x: docs,
        'context': lambda x: concatenated_content,
        'question': itemgetter('question'),
    }
    | prompt
    | llm_with_tool
    | parser_tool
)

In [8]:
answer = chain.invoke({"question": "How to create a RAG chain in LCEL?"})

In [ ]:
chain.invoke({"question": "What is a LCEL?"})

In [9]:
print(answer)

[code(prefix='Creating a Retrieval-Augmented Generation (RAG) chain in LangChain Expression Language (LCEL) involves composing a chain that retrieves relevant documents and then generates a response based on those documents. The process typically involves the following steps: 1. Define the retriever to fetch relevant documents. 2. Define the LLM to generate a response based on the retrieved documents. 3. Compose these steps into a single chain. Below is an example of how to create a RAG chain in LCEL.', imports='from langchain import LCEL, Retriever, LLM', code='# Define the retriever step\nretriever = Retriever(\n    name="document_retriever",\n    params={\n        "index": "my_document_index",\n        "query": "{input_query}"\n    }\n)\n\n# Define the LLM step\nllm = LLM(\n    name="response_generator",\n    params={\n        "model": "gpt-3.5-turbo",\n        "prompt": "Based on the following documents: {retrieved_documents}, answer the question: {input_query}"\n    }\n)\n\n# Comp

In [20]:
answer[0]

code(prefix='Creating a Retrieval-Augmented Generation (RAG) chain in LangChain Expression Language (LCEL) involves composing a chain that retrieves relevant documents and then generates a response based on those documents. The process typically involves the following steps: 1. Define the retriever to fetch relevant documents. 2. Define the LLM to generate a response based on the retrieved documents. 3. Compose these steps into a single chain. Below is an example of how to create a RAG chain in LCEL.', imports='from langchain import LCEL, Retriever, LLM', code='# Define the retriever step\nretriever = Retriever(\n    name="document_retriever",\n    params={\n        "index": "my_document_index",\n        "query": "{input_query}"\n    }\n)\n\n# Define the LLM step\nllm = LLM(\n    name="response_generator",\n    params={\n        "model": "gpt-3.5-turbo",\n        "prompt": "Based on the following documents: {retrieved_documents}, answer the question: {input_query}"\n    }\n)\n\n# Compo

In [26]:
print(answer[0].code)

# Define the retriever step
retriever = Retriever(
    name="document_retriever",
    params={
        "index": "my_document_index",
        "query": "{input_query}"
    }
)

# Define the LLM step
llm = LLM(
    name="response_generator",
    params={
        "model": "gpt-3.5-turbo",
        "prompt": "Based on the following documents: {retrieved_documents}, answer the question: {input_query}"
    }
)

# Compose the RAG chain
rag_chain = LCEL(
    steps=[
        retriever,
        llm
    ],
    input_schema={
        "type": "object",
        "properties": {
            "input_query": {"type": "string"}
        },
        "required": ["input_query"]
    },
    output_schema={
        "type": "object",
        "properties": {
            "response": {"type": "string"}
        }
    }
)

# Example usage
input_data = {"input_query": "What is the capital of France?"}
response = rag_chain.run(input_data)
print(response)


In [24]:
print(answer[0].imports)

from langchain import LCEL, Retriever, LLM


In [25]:
print(answer[0].prefix)

Creating a Retrieval-Augmented Generation (RAG) chain in LangChain Expression Language (LCEL) involves composing a chain that retrieves relevant documents and then generates a response based on those documents. The process typically involves the following steps: 1. Define the retriever to fetch relevant documents. 2. Define the LLM to generate a response based on the retrieved documents. 3. Compose these steps into a single chain. Below is an example of how to create a RAG chain in LCEL.


## State

In [34]:
from typing import Dict, TypedDict
from langchain_core.messages import BaseMessage

class GraphState(TypedDict):
    """
    Represents the state of the graph.
    
    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [36]:
a = GraphState(keys={"key1": "value1", "key2": "value2"})
print(a)

{'keys': {'key1': 'value1', 'key2': 'value2'}}


In [42]:

from operator import itemgetter
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_openai import ChatOpenAI
#from langchain_community.chat_models import ChatAnyscale
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool

def generate(state):
    """
    Generate a code solution based on LCEL docs and the input question 
    with optional feedback from code execution tests 

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    
    ## State
    state_dict = state["keys"]
    question = state_dict["question"]
    
    ## Context 
    # LCEL docs
    url = "https://python.langchain.com/docs/expression_language/"
    loader = RecursiveUrlLoader(
        url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()
    
    # LCEL w/ PydanticOutputParser (outside the primary LCEL docs)
    url = "https://python.langchain.com/docs/modules/model_io/output_parsers/quick_start"
    loader = RecursiveUrlLoader(
        url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs_pydantic = loader.load()
    
    # LCEL w/ Self Query (outside the primary LCEL docs)
    url = "https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/"
    loader = RecursiveUrlLoader(
        url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs_sq = loader.load()
    
    # Add 
    docs.extend([*docs_pydantic, *docs_sq])
    
    # Sort the list based on the URLs in 'metadata' -> 'source'
    d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
    d_reversed = list(reversed(d_sorted))
    
    # Concatenate the 'page_content' of each sorted dictionary
    concatenated_content = "\n\n\n --- \n\n\n".join(
        [doc.page_content for doc in d_reversed]
    )
    
    ## Data model
    class code(BaseModel):
        """Code output"""
        prefix: str = Field(description="Description of the problem and approach")
        imports: str = Field(description="Code block import statements")
        code: str = Field(description="Code block not including import statements")
    
    ## LLM
    model = ChatOpenAI(temperature=0, model="gpt-4o", streaming=True)
    
    # Tool
    code_tool_oai = convert_to_openai_tool(code)
    
    # LLM with tool and enforce invocation
    llm_with_tool = model.bind(
        tools=[code_tool_oai],
        tool_choice={"type": "function", "function": {"name": "code"}},
    )
    
    # Parser
    parser_tool = PydanticToolsParser(tools=[code])
    
    ## Prompt
    template = """You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
        Here is a full set of LCEL documentation: 
        \n ------- \n
        {context} 
        \n ------- \n
        Answer the user question based on the above provided documentation. \n
        Ensure any code you provide can be executed with all required imports and variables defined. \n
        Structure your answer with a description of the code solution. \n
        Then list the imports. And finally list the functioning code block. \n
        Here is the user question: \n --- --- --- \n {question}"""

    ## Generation
    if "error" in state_dict:
        print("---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---")
        
        error = state_dict["error"]
        code_solution = state_dict["generation"]
        
        # Udpate prompt 
        addendum = """  \n --- --- --- \n You previously tried to solve this problem. \n Here is your solution:  
                    \n --- --- --- \n {generation}  \n --- --- --- \n  Here is the resulting error from code 
                    execution:  \n --- --- --- \n {error}  \n --- --- --- \n Please re-try to answer this. 
                    Structure your answer with a description of the code solution. \n Then list the imports. 
                    And finally list the functioning code block. Structure your answer with a description of 
                    the code solution. \n Then list the imports. And finally list the functioning code block. 
                    \n Here is the user question: \n --- --- --- \n {question}"""
        template = template +  addendum

        # Prompt 
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "question", "generation", "error"],
        )
        
        # Chain
        chain = (
            {
                "context": lambda x: concatenated_content,
                "question": itemgetter("question"),
                "generation": itemgetter("generation"),
                "error": itemgetter("error"),
            }
            | prompt
            | llm_with_tool 
            | parser_tool
        )

        code_solution = chain.invoke({"question":question,
                                      "generation":str(code_solution[0]),
                                      "error":error})
                
    else:
        print("---GENERATE SOLUTION---")
        
        # Prompt 
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        )

        # Chain
        chain = (
            {
                "context": lambda x: concatenated_content,
                "question": itemgetter("question"),
            }
            | prompt
            | llm_with_tool 
            | parser_tool
        )

        code_solution = chain.invoke({"question":question})
    
    return {"keys": {"generation": code_solution, "question": question}}

def check_code_imports(state):
    """
    Check imports

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """
    
    ## State
    print("---CHECKING CODE IMPORTS---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    imports = code_solution[0].imports

    try:        
        # Attempt to execute the imports
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        # Catch any error during execution (e.g., ImportError, SyntaxError)
        error = f"Execution error: {e}"
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most recent run error --- \n" + error     
    else:
        print("---CODE IMPORT CHECK: SUCCESS---")
        # No errors occurred
        error = "None"

    return {"keys": {"generation": code_solution, "question": question, "error": error}}

def check_code_execution(state):
    """
    Check code block execution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """
    
    ## State
    print("---CHECKING CODE EXECUTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    imports = code_solution[0].imports
    code = code_solution[0].code
    code_block = imports +"\n"+ code

    try:        
        # Attempt to execute the code block
        exec(code_block)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        # Catch any error during execution (e.g., ImportError, SyntaxError)
        error = f"Execution error: {e}"
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most recent run error --- \n" + error  
    else:
        print("---CODE BLOCK CHECK: SUCCESS---")
        # No errors occurred
        error = "None"

    return {"keys": {"generation": code_solution, "question": question, "error": error}}


### Edges

def decide_to_check_code_exec(state):
    """
    Determines whether to test code execution, or re-try answer generation.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO TEST CODE EXECUTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    error = state_dict["error"]

    if error == "None":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TEST CODE EXECUTION---")
        return "check_code_execution"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"

def decide_to_finish(state):
    """
    Determines whether to finish.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO TEST CODE EXECUTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    error = state_dict["error"]

    if error == "None":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TEST CODE EXECUTION---")
        return "end"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"

In [43]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate) # generate solution
workflow.add_node("check_code_imports", check_code_imports) # check imports
workflow.add_node("check_code_execution", check_code_execution) # check code execution

# Build graph
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check_code_imports")
workflow.add_conditional_edges(
    "check_code_imports", 
    decide_to_check_code_exec,
    {   
    "check_code_execution": "check_code_execution", 
    "generate": "generate"},
)

workflow.add_conditional_edges(
    "check_code_execution", 
    decide_to_finish,
    {   
    "end": END, 
    "generate": "generate"},
)

# Compile
app = workflow.compile()

In [44]:
question = "I am passing text key 'foo' to my prompt and want to process it with a function, process_text(...), prior to the prompt."

config = {"recursion_limit": 50}
app.invoke({"keys": {"question": question, "iterations": 0}}, config=config)

---GENERATE SOLUTION---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: RE-TRY SOLUTION---
---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: RE-TRY SOLUTION---
---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
---CODE BLOCK CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: RE-TRY SOLUTION---
---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---
---CHECKING CODE IMPORTS---
---CODE IMPORT CHECK: SUCCESS---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: TEST CODE EXECUTION---
---CHECKING CODE EXECUTION---
---CODE BLOCK CHECK: FAILED---
---DECIDE TO TEST CODE EXECUTION---
---DECISION: RE-TRY SOLUTION---
---RE-GENERATE SOLUTION w/ ERROR FEEDBACK---
---CHECKING CODE

GraphRecursionError: Recursion limit of 50 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.